<a href="https://colab.research.google.com/github/chunsengwong/SER-PDSSM/blob/main/PDSSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import io
import pandas as pd
from google.colab import files
from matplotlib.pyplot import fill
import pickle
import librosa
from librosa import feature
import soundfile
import numpy as np
!pip install pickle
!pip install pydub
import os
#install google cloud speech
!pip install --upgrade google-cloud-speech
from google.cloud import speech
#set google environment & credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS']= '/content/focus-cache-352904-21b9fded3bd2.json'
speech_client=speech.SpeechClient()
from pydub import AudioSegment
from pydub.playback import play
from IPython.display import Audio, display

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [ ]:
# Load the audio file
path="/content/Harvard list 09.wav"
audio = AudioSegment.from_wav(path)

# Set the desired sample rate (in Hz)
new_sample_rate = 16000

# Resample the audio to the desired sample rate
resampled_audio = audio.set_frame_rate(new_sample_rate)

# Convert the audio to the desired sample width
resampled_audio = audio.set_sample_width(2)

# Export the resampled audio to a new WAV file
resampled_audio.export('/content/output.wav', format='wav')
with open('/content/output.wav', "rb") as audio_file:
    wavfile = audio_file.read()

#import for audio functions
audioWav = speech.RecognitionAudio(content=wavfile)

#create class for audio
class Audioseg:
  def __init__(self, path, start, end):
    self.starttime=start*1000
    self.endtime = end*1000
    self.path=path

#configure wav file
configWav = speech.RecognitionConfig(
       encoding = 'LINEAR16',
        language_code = 'en_us',
        sample_rate_hertz = 48000,
        #audio_channel_count = 2,
        enable_automatic_punctuation=True,
        enable_word_time_offsets=True
)

#trascribe function
def printTranscipt(path, response):
  #display transcript with punctuation marks (automated)
  print("Original audio: ")
  display(AudioSegment.from_wav(path))

  #prompt user to wait during runtime
  print("Waiting for operation to complete...\n\n-------------------------------------")
  print("Transcript: \n")
  for result in response.results:
    print("{}".format(result.alternatives[0].transcript))

In [ ]:
#transcript and automated punctuation added
response = speech_client.long_running_recognize(config=configWav, audio=audioWav)
done=0
#a list to store audio fragments
audioList=[]
AudioS=[]
sentences=[]
sentencesWords=""
response = response.result(timeout=90)
#call printTrascript function
printTranscipt(path,response)


#segment the speech based on the indicators
print("\nindicators: '.' , '!' , '?'")
for i, result in enumerate(response.results):
    alternative = result.alternatives[0]
    for word_info in alternative.words:
            word = word_info.word
            start_time = word_info.start_time
            end_time = word_info.end_time
            sentencesWords=sentencesWords+" "+word

            if(done==0):
                start=start_time.total_seconds()
                done=1


            if word[-1]=='.' or word[-1]=='!' or word[-1]=='?' :
                  done=0
                  sentences.append(sentencesWords)
                  sentencesWords=""
                  AudioS.append(Audioseg(path, start, end_time.total_seconds()))
                  newAudio = AudioSegment.from_wav(AudioS[len(AudioS)-1].path)
                  newAudio = newAudio[AudioS[len(AudioS)-1].starttime:AudioS[len(AudioS)-1].endtime]
                  audioList.append(newAudio)

print(f"\n\n\nAudio fragments (total {(len(audioList))}):")
j=0
emotionData=[]
#display result
for i in audioList:
  print("Sentence " , (j+1) , ":", sentences[j])
  display(i)
  i.export("temp.wav", format="wav")
  print( "\n", (AudioS[j].starttime/1000) , ": ", (AudioS[j].endtime/1000), " (second)")
  j=j+1
  print("\n")

Original audio: 


Waiting for operation to complete...

-------------------------------------
Transcript: 

Harvard list number 9
 The Navy attacked the big task force.
 See the glaring at the scared Mouse.
 There are more than two factors hair.
 The Hat brim with wide and two droopy.
 The lawyer try to lose his case.
 the Girl Scout around the fence post
 cut the pie into large parts.
 men Strife, but seldom get rich
 always close the Barn Door tight.
 He lay prone and hardly move the limb.

indicators: '.' , '!' , '?'



Audio fragments (total 8):
Sentence  1 :  Harvard list number 9 The Navy attacked the big task force.



 5.0 :  11.1  (second)


Sentence  2 :  See the glaring at the scared Mouse.



 13.5 :  15.8  (second)


Sentence  3 :  There are more than two factors hair.



 18.2 :  20.2  (second)


Sentence  4 :  The Hat brim with wide and two droopy.



 23.2 :  25.5  (second)


Sentence  5 :  The lawyer try to lose his case.



 28.0 :  30.0  (second)


Sentence  6 :  the Girl Scout around the fence post cut the pie into large parts.



 32.7 :  40.1  (second)


Sentence  7 :  men Strife, but seldom get rich always close the Barn Door tight.



 42.4 :  49.3  (second)


Sentence  8 :  He lay prone and hardly move the limb.



 51.8 :  54.1  (second)


